In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import re
import glob
import math
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType
from datetime import datetime, timedelta

In [2]:
spark = (
    SparkSession.builder.appName('Specific Analysis')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/10/01 15:07:42 WARN Utils: Your hostname, DESKTOP-F216TKE resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/01 15:07:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/01 15:07:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/01 15:07:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
transactions = spark.read.parquet(".././data/curated/full_transactions2")

In [4]:
transactions.show()

+--------+--------------+------------+-------+-----------+------------------+--------------------+------------------+-----+------+-----------------+--------------------+--------------------+--------------------+-----+-----+--------------+------------------+-------------------------+-------------------------+----------------------+------+-------+-------+
|postcode|order_datetime|merchant_abn|user_id|consumer_id|      dollar_value|            order_id|     consumer_name|state|gender|fraud_probability|       merchant_name|                tags|               tags1|tags2|tags3|merchant_fraud|Median_age_persons|Median_tot_fam_inc_weekly|Median_tot_hhd_inc_weekly|Average_household_size| Tot_M|  Tot_P|  Tot_F|
+--------+--------------+------------+-------+-----------+------------------+--------------------+------------------+-----+------+-----------------+--------------------+--------------------+--------------------+-----+-----+--------------+------------------+-------------------------+-----

In [10]:
tagsdf = transactions.groupBy('tags1').agg(F.count('*')).sort('count(1)')

In [11]:
tagsdfcsv = tagsdf.toPandas()
tagsdfcsv.to_csv('tags.csv')

In [14]:
top_tags = tagsdf.tail(20)

The following tags are the most popular 

In [16]:
top_tags

[Row(tags1='[antique shops - sales, repairs, and restoration services]', count(1)=171323),
 Row(tags1='[bicycle shops - sales and service]', count(1)=220834),
 Row(tags1='[hobby, toy and game shops]', count(1)=231032),
 Row(tags1='[health and beauty spas]', count(1)=243370),
 Row(tags1='[motor vehicle supplies and new parts]', count(1)=289514),
 Row(tags1='[music shops - musical instruments, pianos, and sheet music]', count(1)=304750),
 Row(tags1='[lawn and garden supply outlets, including nurseries]', count(1)=336254),
 Row(tags1='[books, periodicals, and newspapers]', count(1)=381316),
 Row(tags1='[shoe shops]', count(1)=402127),
 Row(tags1='[furniture, home furnishings and equipment shops, and manufacturers, except appliances]', count(1)=447923),
 Row(tags1='[artist supply and craft shops]', count(1)=464672),
 Row(tags1='[computer programming , data processing, and integrated systems design services]', count(1)=499846),
 Row(tags1='[computers, computer peripheral equipment, and soft